In [1]:
%aiida

Loaded AiiDA DB environment - profile name: default.

In [2]:
try:
    import aiida_sssp_workflow
except ModuleNotFoundError as err:
    print("######################################")
    print("### Doing initial one-time install ###")
    ! pip install -e .
    print("######################################")
    ! reentry scan

In [3]:
import ipywidgets as ipw

from pseudo import PseudoSelectionWidget
from delta_factor import ComputeDeltaFactorWidget
from process import ProcessStatusWidget
from aiidalab_widgets_base import ProcessFollowerWidget, ProgressBarWidget, ProcessReportWidget, ProcessCallStackWidget
from process import ProcessInputsWidget, ProcessOutputsWidget

## Step 1: Upload the UPF file

In [4]:
pseudo_selection_step = PseudoSelectionWidget()
display(pseudo_selection_step)

PseudoSelectionWidget(children=(Label(value='Select a pseudopotential from one of the following sources and th…

## Step 2: setting parameters and submit

In [5]:
compute_delta_factor_step = ComputeDeltaFactorWidget()

ipw.dlink((pseudo_selection_step, 'confirmed_pseudo'), (compute_delta_factor_step, 'input_pseudo'))

# Reset all subsequent steps in case that a new pseudo is selected
def _observe_pseudo_selection(change):
    with pseudo_selection_step.hold_sync():
        if pseudo_selection_step.confirmed_pseudo is not None and \
                pseudo_selection_step.confirmed_pseudo != change['new']:
            pseudo_selection_step.confirmed_pseudo = change['new']
pseudo_selection_step.observe(_observe_pseudo_selection, 'pseudo')


/opt/conda/lib/python3.7/site-packages/aiida/orm/computers.py:742: AiidaDeprecationWarning: this property is deprecated, use the `label` property instead
  warnings.warn('this property is deprecated, use the `label` property instead', AiidaDeprecationWarning)  # pylint: disable=no-member


In [6]:
display(compute_delta_factor_step)

ComputeDeltaFactorWidget(children=(Label(value='Specify the parameters and options for the calculation and the…

## Step3: Check the state of the process

In [7]:
process_status = ProcessStatusWidget()

ipw.dlink((compute_delta_factor_step, 'process'), (process_status, 'process'))

In [8]:
process_status.process

In [9]:
display(process_status)

ProcessStatusWidget(children=(ProgressBarWidget(children=(HTML(value='N/A', description='Calculation state:', …

## Step4: Check the output of workflow

In [10]:
process_input = ProcessInputsWidget()
process_output = ProcessOutputsWidget()
process_follower = ProcessFollowerWidget(
    followers=[ProgressBarWidget(), ProcessReportWidget(), ProcessCallStackWidget()], path_to_root='../', 
    update_interval=2)

ipw.dlink((compute_delta_factor_step, 'process'), (process_input, 'process'))
ipw.dlink((compute_delta_factor_step, 'process'), (process_output, 'process'))
ipw.dlink((compute_delta_factor_step, 'process'), (process_follower, 'process'))

In [11]:
display(process_input)

ProcessInputsWidget(children=(HBox(children=(Dropdown(description='Select input:', options=(('Select input', '…

In [12]:
display(process_output)

ProcessOutputsWidget(children=(HBox(children=(Dropdown(description='Select outputs:', options=(('Select output…

In [13]:
from aiida.orm import load_node
compute_delta_factor_step.process = load_node(65)

In [14]:
process_input.process

<WorkChainNode: uuid: a772b6d2-db3b-4f79-bce3-caa84c4acd79 (pk: 65) (aiida.workflows:sssp_workflow.delta_factor)>

In [15]:
compute_delta_factor_step.process

<WorkChainNode: uuid: a772b6d2-db3b-4f79-bce3-caa84c4acd79 (pk: 65) (aiida.workflows:sssp_workflow.delta_factor)>

In [16]:
display(process_follower)
process_follower.follow(detach=True)

ProcessFollowerWidget(children=(HTML(value=''), VBox(children=(HTML(value='<h2><b>Progress Bar</b></h2>'), Pro…